# Sample Jupyter Notebook in Clojure for JQuants

Start by installing clojupyter. using:

```bash
# https://cljdoc.org/d/nubank/clojupyter/0.2.2/doc/readme#installation
git clone https://github.com/clojupyter/clojupyter    
make && make install   
```
Then you can start the jupyter console as usual:
``` bash
jupyter-console
```

In [ ]:
(require '[clojupyter.misc.helper :as helper])
; (helper/add-dependencies '[metasoarous/oz "2.0.0-alpha5"])
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(helper/add-dependencies '[net.clojars.hellonico/jquants-api-jvm "0.2.11"])

In [12]:
(require '[hellonico.jquants-api :as api])
(require '[oz.notebook.clojupyter :as oz])

nil

# Draw a simple graph with oz in Jupyter

In [13]:
(defn datapt [[s n]] {:Index s, :Value n})
(defn graph  [vs]    {:data {:values vs}, 
                      :mark :line
                      :encoding {:x {:field :Index}
                                 :y {:field :Value}}})
(def graph-it (comp graph (partial map datapt) (partial apply map vector)));

In [14]:
(defn raw-data [] (map #(+ % (rand-int 25)) (range)))

#'user/raw-data

In [15]:
(->> [(range) (take 150 (raw-data))]
     graph-it
     oz/view!)

# Get the quotes from jquants and draw them using oz

In [25]:
; (def data (:daily_quotes (api/daily-fuzzy {:CompanyNameEnglish "M3"})))
(def data (:daily_quotes (api/daily {:code 2413 :from 20220301 :to 20220328})))

(defn average [coll]
  (/ (reduce + coll)
     (count coll)))

(defn ma [period coll]
  (lazy-cat (repeat (dec period) nil)
            (map average (partition period 1  coll))))
(def ma-5 (partial ma 5))

(def _data (map #(assoc %1 :ma %2) data (ma-5 (map :High data))))


{"daily_quotes": [{"Code": "24130", "Close": 4512.0, "Date": "20220301", "AdjustmentHigh": 4534.0, "Volume": 4800600.0, "TurnoverValue": 21391696600.0, "AdjustmentClose": 4512.0, "AdjustmentLow": 4270.0, "Low": 4270.0, "High": 4534.0, "Open": 4315.0, "AdjustmentOpen": 4315.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 4800600.0}, {"Code": "24130", "Close": 4419.0, "Date": "20220302", "AdjustmentHigh": 4588.0, "Volume": 4432200.0, "TurnoverValue": 19702595700.0, "AdjustmentClose": 4419.0, "AdjustmentLow": 4373.0, "Low": 4373.0, "High": 4588.0, "Open": 4455.0, "AdjustmentOpen": 4455.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 4432200.0}, {"Code": "24130", "Close": 4549.0, "Date": "20220303", "AdjustmentHigh": 4666.0, "Volume": 5004700.0, "TurnoverValue": 22920157700.0, "AdjustmentClose": 4549.0, "AdjustmentLow": 4510.0, "Low": 4510.0, "High": 4666.0, "Open": 4595.0, "AdjustmentOpen": 4595.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 5004700.0}, {"Code": "24130", "Close": 4362.0,

#'user/_data

In [26]:
(def data-plot
  {:data {:values _data}
   :encoding {:x {:field "Date" :type "ordinal"}
              :y {:field "ma" :type "quantitative"}
              :color {:field "Code" :type "nominal"}}
   :mark "line"}
  )

(def viz
  [:div
   [:h1 "5 days Moving average"]
   [:p "Entity code 24130"]
   [:p "from 20220301 to 20220328"]
   [:vega-lite data-plot]
   ;[:h2 "If ever, oh ever a viz there was, the vizard of oz is one because, because, because..."]
   [:p (str (java.util.Date.))]
   ])

#'user/viz

In [24]:
(oz/view! viz)

5 days Moving average Entity code 24130 from 20220301 to 20220328 
 
 
 
 
 Wed Oct 05 11:21:55 JST 2022